<a href="https://colab.research.google.com/github/Rainniee/Neural-Networks-AI/blob/master/Chatbot%20CNN%20Preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import os
import hashlib
import struct
import subprocess
import collections

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!unzip "drive/My Drive/Assignment 4 Chatbot/finished_files.zip"

Archive:  drive/My Drive/Assignment 4 Chatbot/finished_files.zip
   creating: finished_files/
  inflating: finished_files/test.bin  
  inflating: finished_files/train.bin  
  inflating: finished_files/val.bin  
  inflating: finished_files/vocab    
   creating: finished_files/chunked/
  inflating: finished_files/chunked/train_127.bin  
  inflating: finished_files/chunked/train_126.bin  
  inflating: finished_files/chunked/train_125.bin  
  inflating: finished_files/chunked/train_124.bin  
  inflating: finished_files/chunked/train_123.bin  
  inflating: finished_files/chunked/train_122.bin  
  inflating: finished_files/chunked/train_121.bin  
  inflating: finished_files/chunked/train_120.bin  
  inflating: finished_files/chunked/train_119.bin  
  inflating: finished_files/chunked/train_118.bin  
  inflating: finished_files/chunked/train_117.bin  
  inflating: finished_files/chunked/train_116.bin  
  inflating: finished_files/chunked/train_115.bin  
  inflating: finished_files/chunked/tr

In [3]:
!unzip "drive/My Drive/Assignment 4 Chatbot/cnn_stories_tokenized.zip"

Archive:  drive/My Drive/Assignment 4 Chatbot/cnn_stories_tokenized.zip
replace cnn_stories_tokenized/daf94a7a113ecd9303f4ab32cea2217d03fe35bb.story? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
get_art_abs("drive/My Drive/Colab Notebooks/data/000cd1ee0098c4d510a03ddc97d11764448ebac2.story")

("kathmandu , nepal -lrb- cnn -rrb- -- a ferocious leopard may have killed 15 people in nepal in a 15-month span , its latest victim a 4-year-old boy that the creature dragged away into the jungle to eat . the head of boy was found in the forest a kilometer from his home saturday morning , said kamal prasad kharel , the police chief of the baitadi district , an area about 600 kilometers -lrb- 373 miles -rrb- west of kathmandu . the grisly discovery , which came after teams of people searched for the child , marks the 15th victim in the past 15 months in that remote district in western nepal . the police chief suspects that a single man-eating leopard is responsible for the deaths . if not , there are at most two of the man-eating creatures around , he believes . maheshwor dhakal , an ecologist at the department of national parks and wildlife conservation in kathmandu , agreed that it is unusual to find more than one or two man-eating animals in one area . most leopards live on wild pre

In [0]:
with open(cnn_content_file,'w',encoding='utf-8') as content, open(cnn_summary_file,'w',encoding='utf-8') as summary:
        for s in cnn_stories:
            s_file = os.path.join(os.path.join(os.getcwd(),cnn_tokenized_stories_dir,s))
            article,abstract = get_art_abs(s_file)
            if(len(article.split())!=0):
              content.write(article.rstrip('\n')+'\n')
              summary.write(abstract.rstrip('\n')+'\n')

In [0]:
article,abstract = get_art_abs("drive/My Drive/Colab Notebooks/data/000cd1ee0098c4d510a03ddc97d11764448ebac2.story")

if(len(article.split())!=0):
  content.write(article.rstrip('\n')+'\n')
  summary.write(abstract.rstrip('\n')+'\n')

NameError: ignored

In [0]:
dm_single_close_quote = u'\u2019' # unicode
dm_double_close_quote = u'\u201d'
END_TOKENS = ['.', '!', '?', '...', "'", "`", '"', dm_single_close_quote, dm_double_close_quote, ")"] # acceptable ways to end a sentence

In [0]:
# We use these to separate the summary sentences in the .bin datafiles
SENTENCE_START = '<s>'
SENTENCE_END = '</s>'

In [0]:
cnn_tokenized_stories_dir = "cnn_stories_tokenized"

In [0]:
# These are the number of .story files we expect there to be in cnn_stories_dir
# num_expected_cnn_stories = 92579

In [0]:
def tokenize_stories(stories_dir, tokenized_stories_dir):
  """Maps a whole directory of .story files to a tokenized version using Stanford CoreNLP Tokenizer"""
  print("Preparing to tokenize %s to %s..." % (stories_dir, tokenized_stories_dir))
  
  stories = os.listdir(stories_dir)
  # make IO list file
  print("Making list of files to tokenize...")
  
  with open("mapping.txt", "w") as f:
    for s in stories:
      f.write("%s \t %s\n" % (os.path.join(stories_dir, s), os.path.join(tokenized_stories_dir, s)))
  command = ['java', 'edu.stanford.nlp.process.PTBTokenizer', '-ioFileList', '-preserveLines', 'mapping.txt']
  print("Tokenizing %i files in %s and saving in %s..." % (len(stories), stories_dir, tokenized_stories_dir))
  
  subprocess.call(command)
  print("Stanford CoreNLP Tokenizer has finished.")
  
  os.remove("mapping.txt")

  # Check that the tokenized stories directory contains the same number of files as the original directory
  #num_orig = len(os.listdir(stories_dir))
  #num_tokenized = len(os.listdir(tokenized_stories_dir))
  
  if num_orig != num_tokenized:
    raise Exception("The tokenized stories directory %s contains %i files, but it should contain the same number as %s (which has %i files). Was there an error during tokenization?" % (tokenized_stories_dir, num_tokenized, stories_dir, num_orig))
  
  print("Successfully finished tokenizing %s to %s.\n" % (stories_dir, tokenized_stories_dir))

In [0]:
def read_text_file(text_file):
  lines = []
  with open(text_file, "r") as f:
    for line in f:
      lines.append(line.strip())
  return lines

def fix_missing_period(line):
  """Adds a period to a line that is missing a period"""
  if "@highlight" in line: return line
  if line=="": return line
  if line[-1] in END_TOKENS: return line
  return line + " ."

In [0]:
def get_art_abs(story_file):
  lines = read_text_file(story_file)

  # Lowercase everything
  lines = [line.lower() for line in lines]

  '''
  Put periods on the ends of lines that are missing them 
  this is a problem in the dataset because many image captions don't end in periods
  consequently they end up in the body of the article as run-on sentences
  '''
  lines = [fix_missing_period(line) for line in lines]

  # Separate out article and abstract sentences
  article_lines = []
  highlights = []
  next_is_highlight = False
  
  for idx,line in enumerate(lines):
    if line == "":
      continue # empty line
      
    elif line.startswith("@highlight"):
      next_is_highlight = True
      
    elif next_is_highlight:
      highlights.append(line)
      
    else:
      article_lines.append(line)

  # Make article into a single string
  article = ' '.join(article_lines)

  # Make abstract into a signle string, putting <s> and </s> tags around the sentences
  abstract = ' '.join(highlights)

  return article, abstract

In [0]:
'''
def check_num_stories(stories_dir, num_expected):
  num_stories = len(os.listdir(stories_dir))
  
  if num_stories != num_expected:
    raise Exception("stories directory %s contains %i files but should contain %i" % (stories_dir, num_stories, num_expected))
    
'''

In [0]:
def create_corpus(cnn_tokenized_stories_dir, cnn_corpus_dir):
    cnn_stories = os.listdir(cnn_tokenized_stories_dir)
    
    cnn_content_file = os.path.join(cnn_corpus_dir,'articles.txt')
    cnn_summary_file = os.path.join(cnn_corpus_dir,'abstracts.txt')

    with open(cnn_content_file,'w',encoding='utf-8') as content, open(cnn_summary_file,'w',encoding='utf-8') as summary:
        for s in cnn_stories:
            s_file = os.path.join(os.path.join(os.getcwd(),cnn_tokenized_stories_dir,s))
            article,abstract = get_art_abs(s_file)
            if(len(article.split())!=0):
              content.write(article.rstrip('\n')+'\n')
              summary.write(abstract.rstrip('\n')+'\n')

In [0]:
cnn_corpus_dir = "cnn_stories_tokenized"

In [0]:
create_corpus(cnn_tokenized_stories_dir, cnn_corpus_dir)

cnn_stories_tokenized/  drive/  finished_files/  sample_data/
